In [4]:
###############################################################################
# Common parameters
###############################################################################
'''
year_plan, month_plan, l_holiday  = 2024, 6, []
year_plan, month_plan, l_holiday  = 2024, 7, [15]
year_plan, month_plan, l_holiday  = 2024, 8, [12]
year_plan, month_plan, l_holiday  = 2024, 9, [16, 23]
year_plan, month_plan, l_holiday  = 2024, 10, [14]
year_plan, month_plan, l_holiday  = 2024, 11, [4]
year_plan, month_plan, l_holiday  = 2024, 12, [28, 29, 30, 31]
year_plan, month_plan, l_holiday  = 2025, 2, [11, 24]
year_plan, month_plan, l_holiday  = 2025, 4, [29]
year_plan, month_plan, l_holiday  = 2025, 5, [5,6]
year_plan, month_plan, l_holiday  = 2025, 6, []
year_plan, month_plan, l_holiday  = 2025, 7, [21]
year_plan, month_plan, l_holiday  = 2025, 8, [11]
'''
year_plan, month_plan, l_holiday  = 2025, 9, [15, 23]
l_date_ect_cancel = []

from script.parameter import *

In [2]:
###############################################################################
# Create Google form
###############################################################################
from script.form import *
d_cal, d_date_duty, s_cnt_duty, s_cnt_class_duty, d_cal_duty, d_form =\
    prepare_form(lp_root, year_plan, month_plan, l_holiday, l_date_ect_cancel, l_day_ect, day_em, l_week_em, l_class_duty, dict_duty, dict_score_duty, dict_duty_jpn, dict_title_duty, dict_class_duty, id_template_form, dict_itemid_form)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=286844707417-54por1avpn0r7ulus5qoa1fc3mb0jjhm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62605%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.body&state=HeKeAvfqTPJOiOoWVFeTvOR2gyNYOu&access_type=offline
Form URL: https://docs.google.com/forms/d/e/1FAIpQLSes6t0fw-_GIP_VUgK-lQtCS642D4ayUe5KMByU4nZBxqe7iA/viewform


In [2]:
###############################################################################
# Collect Google form response
###############################################################################
from script.collect import *
str_member_missing, str_mail_missing, d_availability, d_info, d_member =\
    collect_availability(lp_root, year_plan, month_plan, dict_jpnday, dict_duty_jpn)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=286844707417-54por1avpn0r7ulus5qoa1fc3mb0jjhm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53078%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.body&state=UXqFX6Tq3WWMcqKzdg9nX3YqROT1zs&access_type=offline
Missing members and emails:
大熊 彩子, 小黒 早紀
okuma.ayako@gmail.com, saki.oguro@gmail.com
Requests:
市橋 香代 いつもありがとうございます。
佐藤 駿一 少なくて大変申し訳ございません。最近子どもの体調があまり良くないため、可能な日程は実家のサポートが得られそうな日だけになってしまっています。ご了承いただけますと幸いです。
佐藤 未悠 Eチームですが諸用のため15日お休みします


In [6]:
###############################################################################
# Optimize assignment count and assign members
###############################################################################
from script.assign import *
# Assignment count optimization
dict_c_diff_score_current = {'ampm': 0.001, 'daynight': 0.001, 'ampmdaynight': 0.001, 'oc': 0.001, 'ect': 0.01}
dict_c_diff_score_total = {'ampm': 0.01, 'daynight': 0.01, 'ampmdaynight': 0.01, 'oc': 0.01, 'ect': 0.1}
#dict_c_diff_score_total = {'ampm': 0.01, 'daynight': 0.01, 'ampmdaynight': 1.0, 'oc': 0.01, 'ect': 0.1}
# Individual assignment
# Parameters for avoiding/penalizing close duties
#dict_closeduty = {'daynight': {'l_duty': ['day', 'ocday', 'night', 'emnight', 'ocnight'], 'thr_hard': 2, 'thr_soft': 5}, # 1: avoid within same day, 2: avoid within 2 continuous days
#                  'ect':      {'l_duty': ['ect'],                                         'thr_hard': 1, 'thr_soft': 4},
#                  'ampm':     {'l_duty': ['am', 'pm'],                                    'thr_hard': 1, 'thr_soft': 2}}
dict_closeduty = {'daynight': {'l_duty': ['day', 'ocday', 'night', 'emnight', 'ocnight'], 'thr_hard': 0, 'thr_soft': 5}, # 1: avoid within same day, 2: avoid within 2 continuous days
                  'ect':      {'l_duty': ['ect'],                                         'thr_hard': 1, 'thr_soft': 4},
                  'ampm':     {'l_duty': ['am', 'pm'],                                    'thr_hard': 1, 'thr_soft': 2}}
#c_assign_suboptimal, c_cnt_deviation, c_closeduty = 0.001, 0.001, 0.1
c_assign_suboptimal, c_cnt_deviation, c_closeduty = 0.00001, 0.1, 0.00001
#c_assign_suboptimal, c_cnt_deviation, c_closeduty = 0.0001, 0.1, 0.01
l_date_duty_fulltime = []
# Count deviation penalization mode. student twice-assignation is limited elsewhere
type_limit = 'soft'  # 'hard': never exceed, 'soft': outlier penalized, 'ignore': no penalty
l_date_duty_skip_manual = []
'''
#l_date_duty_skip_manual = ['23_'] # All duties starting with 23_
#l_date_duty_skip_manual = ['23_am']
'''
d_assign, d_assign_date_print, d_assign_member, d_deviation, d_score_print, d_closeduty, d_deviation_summary, d_assign_date_duty =\
    optimize_count_and_assign(lp_root, year_plan, month_plan, year_start, month_start, l_class_duty, dict_c_diff_score_current,
                              dict_c_diff_score_total, l_date_duty_skip_manual, dict_closeduty, ll_avoid_adjacent, l_title_fulltime,
                              l_date_duty_fulltime, type_limit, c_assign_suboptimal, c_cnt_deviation, c_closeduty,
                              dict_score_duty, dict_score_class, dict_class_duty)

Assignment count optimization
------------------------------------------------------------
Done, losses: 6.59(non-OC), 0.29(OC)
Member assignment optimization
------------------------------------------------------------
No member available for: ['3_pm', '12_pm', '20_ocnight', '22_ocnight']
of which ['3_pm', '12_pm'] are not OC
Manually assigned member(s) for: ['15_day']
15_day  manually set to  20.0
In total, skipping assignment for: ['3_pm', '12_pm']
------------------------------------------------------------
Result
------------------------------------------------------------
Done, losses: 1.0 = 0.0 + 1.0 + 0.0
(Total = Suboptimality + Count Deviation + Close duty)
------------------------------------------------------------
Deviation from target:
------------------------------------------------------------
  id_member name_jpn    class_duty  deviation_exact  deviation_hard
0         3       近藤        oc_tot               -1               0
1         6       池亀  daynight_tot         

In [ ]:
###############################################################################
# Notify Google calendar
###############################################################################
from script.notify import *
update_calendar(lp_root, year_plan, month_plan, id_calendar, dict_time_duty, t_sleep)

In [7]:
###############################################################################
# Collect replacement application
###############################################################################
from script.replace import *
d_replace_checked = check_replacement(lp_root, year_plan, month_plan)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=286844707417-54por1avpn0r7ulus5qoa1fc3mb0jjhm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53612%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.body&state=onUFBOI3pXBF2IkvvWAL2oyFN6QA7F&access_type=offline
Replacing:
  month date   duty before after
0     9    8    ect     熊倉    南學
1     9   10  night     清田    池亀


In [8]:
###############################################################################
# Apply checked replacement plan
###############################################################################
d_assign, d_assign_date_print, d_assign_member, d_deviation, d_deviation_summary, d_score_current, d_score_total, d_score_print =\
    replace_assignment(lp_root, year_plan, month_plan, dict_score_duty, l_class_duty, d_replace_checked)